In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

c:\Users\nolan\anaconda3\envs\sp25_taxi\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current UTC hour (ceiled): 2025-02-22 23:00:00+00:00


In [4]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [5]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-02-22 17:14:10,695 INFO: Initializing external client
2025-02-22 17:14:10,696 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:14:11,705 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635


In [6]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.36s) 


In [7]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,193,2.0,2025-02-22 23:00:00+00:00
1,125,27.0,2025-02-22 23:00:00+00:00
2,72,0.0,2025-02-22 23:00:00+00:00
3,20,0.0,2025-02-22 23:00:00+00:00
4,216,1.0,2025-02-22 23:00:00+00:00
...,...,...,...
252,222,0.0,2025-02-22 23:00:00+00:00
253,122,0.0,2025-02-22 23:00:00+00:00
254,62,0.0,2025-02-22 23:00:00+00:00
255,88,14.0,2025-02-22 23:00:00+00:00


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  257 non-null    int32                  
 1   predicted_demand    257 non-null    float64                
 2   pickup_hour         257 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 5.1 KB


In [9]:
from datetime import datetime, timezone, timedelta

def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-02-22 17:15:33,409 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-22 17:15:33,419 INFO: Initializing external client
2025-02-22 17:15:33,420 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-22 17:15:34,227 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Current UTC time: 2025-02-22 22:15:33.409389+00:00
Next hour: 2025-02-22 23:00:00+00:00
Querying for date range: 2025-02-22 to 2025-02-23
Filtering for hour: 2025-02-22 23:00
Found 257 records


In [10]:
now = datetime.now(timezone.utc)

In [11]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,193,2.0,2025-02-22 23:00:00+00:00
1,125,27.0,2025-02-22 23:00:00+00:00
2,72,0.0,2025-02-22 23:00:00+00:00
3,20,0.0,2025-02-22 23:00:00+00:00
4,216,1.0,2025-02-22 23:00:00+00:00
...,...,...,...
252,222,0.0,2025-02-22 23:00:00+00:00
253,122,0.0,2025-02-22 23:00:00+00:00
254,62,0.0,2025-02-22 23:00:00+00:00
255,88,14.0,2025-02-22 23:00:00+00:00
